YOLO to CSV Convert

inputs - Directory of Yolo annotation text files
outputs - a CSV file of the annotations with one annotation per line. 

CSV line information in output

filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
greater_sage-grouse-9696a5ff2c77470862d56159801e08b398e49b10e8d628717185b6900bb031e5.jpg,2226589,"{'width': 4472, 'height': 3470}",1,0,"{'name': 'rect', 'x': 1862, 'y': 524, 'width': 2103, 'height': 2436}",{'category': {'5': True}}

CSV line information needed for the annotator. 

filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
greater_sage-grouse-9696a5ff2c77470862d56159801e08b398e49b10e8d628717185b6900bb031e5.jpg,2226589,"{""width"":4472,""height"":3470}",1,0,"{""name"":""rect"",""x"":1862,""y"":524,""width"":2103,""height"":2436}","{""category"":{""5"":true}}"

Differences:
replace all single quotes ('') with TWO double quotes ("""") 
Replace True with true and probably False with false for the category 
Everything should be lower case
Place double quotes around the category

Feel free to try any combination of these fixes, but I havent had a chance go make them 
in the actual convert. I jut edit the file using find and replace.

In [1]:
import os
import json
import cv2
import csv
import numpy as np
from typing import List, Dict

annotation_dir = "/Users/brosso/Documents/personal_code/CARL/annotation_text_files/"
image_dir = "/Users/brosso/Documents/personal_code/CARL/data2/"
output_file = "./traffic_annotations_data2.csv"

file_counter = 0

# Map class IDs to class names
# Doesnt actually get used. I just like to have it places to read. 

class_map = {
    0: "vehicle",
    # 1: "Lower Sensitive",
    # 2: "Right Arm",
    # 3: "Left Arm",
    # 4: "Right Ankle",
    # 5: "Left Ankle",
    # 6: "Right Leg",
    # 7: "Left Leg",
    # 8: "Torso",
    # 9: "Threat",
    # 10: "Upper Sensitive",
    # 11: 'RIght Wrist', 
    # 12: 'Right Knee', 
    # 13: 'Left Shoulder'
}

###########################################################
# Convert each yolo line to a CSV line.
# Inputs - file path, image width and image height
# Outputs - annotation information for the box. 
###########################################################
def yolo_to_csv(file_path: str,  width: int, height: int) -> List[Dict]:
    annotations = []
    attributes = []
    region_count = 0
    
    # open the yolo text file. 
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            
            # convert the yolo box information (center point, height, width)
            # into CSV box information (bottom left point, height, width)
            # class_id, x_center, y_center, w, h = map(float, line.strip().split())
            if len(line.strip().split()) == 5:
                class_id, x_center, y_center, w, h = map(float, line.strip().split())
                class_id = "0"  # reassigning this to make my life easier later
                x_center *= width
                y_center *= height
                w *= width
                h *= height
                x_min = int(x_center - w / 2)
                y_min = int(y_center - h / 2)

                # CSV annotation information
                annotation = {
                    "name": "rect",
                    "x": int(x_min),
                    "y": int(y_min),
                    "width": int(w),
                    "height": int(h)
                }
            elif len(line.strip().split()) > 5:
                line_split = line.strip().split()
                class_id = line_split.pop(0)

                all_x = []
                all_y = []
                for i in range(0,len(line_split),2):
                    x = line_split.pop(0)
                    y = line_split.pop(0)
                    x = int(float(x) * width)
                    y = int(float(y) * height)
                    all_x.append(x)
                    all_y.append(y)
                
                annotation = {
                    "name": "polygon",
                    "all_points_x": all_x,
                    "all_points_y": all_y
                }

            # label ID
            # region_attribute = {"category":{str(int(class_id)):True}}
            region_attribute = {"category":{str(int(class_id)):True}}
        
            region_count += 1
            annotations.append(annotation)
            attributes.append(region_attribute)

    if len(annotations) == 0:
        annotations.append({})
        attributes.append({})
    return annotations, attributes, region_count, 

csv_annotations = []
csv_attributes = []
csv_images = []
image_id = 0
row_count = 0

# CSV file fieldnames for the header
field_names = ["filename", 
               "file_size", 
               "file_attributes", 
               "region_count", 
               "region_id", 
               "region_shape_attributes", 
               "region_attributes"]

# Open a CSV file output file
with open(output_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)
    writer.writeheader()
    annotation_files = os.listdir(annotation_dir)
    annotation_files.sort()
    # iterate over all files in the YOLO text directory. 
    for filename in annotation_files.copy():
        if filename.endswith('.txt'):
            
            annotation_file = os.path.join(annotation_dir, filename)
            image_file = os.path.join(image_dir, filename.replace('.txt', '.jpg'))
            
            # get image information
            image = cv2.imread(image_file)
            height, width = image.shape[:2]
            size = os.stat(image_file).st_size
            
            file_attributes = {}
            # file_attributes = {"width":width,"height":height}
            
            # get the bbox information from the yolo file
            annotations, attributes, region_count = yolo_to_csv(
                annotation_file, 
                width, 
                height)

            file_counter += 1
            region_id = 0
            # write all the CSV bbox to the CSV
            for i in range(region_count):
                row = {"filename":filename.replace('.txt', '.jpg'), 
                       "file_size":size, 
                       "file_attributes":file_attributes, 
                       "region_count":region_count, 
                       "region_id":i, 
                       "region_shape_attributes":str(annotations[i]).replace("\'","\""), 
                       "region_attributes":str(attributes[i]).replace("\'","\"").replace("True", "true").replace("False","false")}
                writer.writerow(row)
            if region_count == 0:
                row = {"filename":filename.replace('.txt', '.jpg'), 
                       "file_size":size, 
                       "file_attributes": str({}), 
                       "region_count":region_count, 
                       "region_id":0, 
                       "region_shape_attributes":str({}), 
                       "region_attributes": str({})}
                writer.writerow(row)
            image_id += 1


CSV to YOLO ConvertThis convert is still a Work in Progress, and needs some imporvementsinputs - CSV file exported from the Annotator tool
outputs - a directory of YOLO annotation text filesWhen importing the annotations to the annotator from COCO or JSON, the rectangle
boxes are converted to polygons. These polygons are not currently supported in 
converting to YOLO text files.

In [ ]:
import os
import json
import cv2
import csv
import numpy as np
from typing import List, Dict
from tqdm import tqdm
import shutil

animal = "bear"
annotation_dir = "/media/hdfs1/projects/hdait/fulldata/whole_datasets/14000_and_14TB_hands_down/labels"
image_dir = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets_handsdown/batch"
input_file = "/home/ahufford/Projects/Utils/difficult_contrast.csv"

file_counter = 0

isExist = os.path.exists(annotation_dir)
if not isExist:
   os.makedirs(annotation_dir)

# Map class IDs to class names
file_map = {
    "_000.png": "-03.png",
    "_001.png": "-07.png",
    "_002.png": "-11.png",
    "_003.png": "-15.png",
    "_004.png": "-19.png",
    "_005.png": "-23.png",
    "_006.png": "-27.png",
    "_007.png": "-31.png",
    "_008.png": "-35.png",
    "_009.png": "-39.png",
    "_010.png": "-43.png",
    "_011.png": "-47.png",
    "_012.png": "-51.png",
    "_013.png": "-55.png",
    "_014.png": "-59.png",
    "_015.png": "-63.png",
}

class_map = {
    0: "FACE",
    1: "JOINT",
    2: "LIMB",
    3: "EXAMPLE"
}

###########################################################
# Convert each yolo line to a CSV line.
# Inputs -  file path to the CSV file
#           directory to save the yolo files in
#           directory the images are located            
# Outputs - None, the function does the file writing. 
# 
# This function will probably change so I can use the 
# text file write instead of text file append. 
#  
###########################################################
def csv_to_yolo(file_path: str, annotation_dir: str, image_dir: str) -> List[Dict]:
    annotations = []
    attributes = []
    field_names = ["filename", 
               "file_size", 
               "file_attributes", 
               "region_count", 
               "region_id", 
               "region_shape_attributes", 
               "region_attributes"]
    
    region_count = 0
    with open(file_path, newline = '') as f:
        reader = csv.DictReader(f, delimiter=',', quotechar='\"')
        line_count = -1
        xmax = 0
        ymax = 0
        for row in tqdm(reader):
            if line_count == 0:
                header = row
                print(header)
            
            
            else:
                text_filename = row["filename"].replace(".png",".txt").replace(".jpeg",".txt")
                with open(os.path.join(annotation_dir, text_filename), "a") as txt_file:
                    if row["region_id"] < row["region_count"]:

                        label_dict = json.loads(row["region_attributes"])
                        shape_dict = json.loads(row["region_shape_attributes"])
                        image_dict = json.loads(row["file_attributes"])
                        
                        try:
                            img_width = image_dict["width"]
                            img_height = image_dict["height"]

                        except KeyError:
                            image_name = os.path.join(image_dir, row["filename"])
                            # image_name = image_name.replace(image_name[-8:], file_map[image_name[-8:]])
                            img = cv2.imread(image_name)
                            img_height, img_width, c = img.shape

                        box = None
                        if shape_dict["name"] == "rect":
                            x_min = shape_dict["x"]
                            y_min = shape_dict["y"]
                            
                            
                            w = shape_dict["width"]
                            h = shape_dict["height"]

                            x = int(x_min + w/2.0)
                            y = int(y_min + h/2.0)

                            dw = 1.0 / img_width
                            dh = 1.0 / img_height
                            
                            # Normalization
                            x = x * dw
                            y = y * dh
                            w = w * dw
                            h = h * dh
                                       
                            bbox = [str(x), str(y), str(w), str(h)]

                            label = 0
                            for item in label_dict["category"]:
                                if item.isnumeric():
                                    if label_dict["category"][str(item)]:
                                        label = int(item)
                                        if label == 1:
                                            continue
                                else:
                                    val_list = list(class_map.values())
                                    label = val_list.index(item)
                            box = [str(label)] + bbox
                        
                        # catch the difference in shapes. 
                        if shape_dict["name"] == "polygon":
                            x_points = list(shape_dict["all_points_x"])
                            y_points = list(shape_dict["all_points_y"])

                            label = 0
                            # item = label_dict["category"]
                            
                            # if isinstance(item, dict):
                            #     for something in item:
                            #         if something.isnumeric():
                            #             if label_dict["category"][str(something)]:
                            #                 label = int(something)
                                            
                            #         else:
                            #             val_list = list(class_map.values())
                            #             label = val_list.index(something)
                            
                            # elif item.isnumeric():
                            #     if label_dict["category"][item]:
                            #         label = int(item) 
                            # else:
                            #     val_list = list(class_map.values())
                            #     label = val_list.index(item)
                            bbox = []
                            for i in range(len(x_points)):
                                bbox.append(str(float(x_points[i] / img_width)))
                                bbox.append(str(float(y_points[i] / img_height)))
                            if label == 1:
                                continue
                            label = 0
                            box = [str(label)] + bbox
                        if box:
                            txt_file.write(" ".join(box) + "\n")
                        box = None
            

        
def copy_files_to_folder(list_of_files, destination_folder):
    for f in tqdm(list_of_files):
        try:
            dest_file = os.path.join(destination_folder, f.split('/')[-1])
            if not os.path.exists(dest_file):
                shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

        
            
csv_annotations = []
csv_attributes = []
csv_images = []
image_id = 0
row_count = 0

csv_to_yolo(input_file, annotation_dir, image_dir)

In [ ]:
csv_file_dir = "/media/bd1/PerimeterSecurity/text_labels/csv_labels"


for filename in os.listdir(csv_file_dir):
    input_file = os.path.join(csv_file_dir, filename)

    print(animal)
    animal = filename.replace(".csv","")
    annotation_dir =  "/media/bd1/PerimeterSecurity/new_labels/" + animal + "/"
    image_dir = "/media/bd1/PerimeterSecurity/images/" + animal + "/"

    isExist = os.path.exists(annotation_dir)
    if not isExist:
        os.makedirs(annotation_dir)

    csv_to_yolo(input_file, annotation_dir, image_dir)

 folders of yolo to csv

In [ ]:
import os
import json
import cv2
import csv
import numpy as np
from typing import List, Dict

annotation_dirs = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets_handsdown/yolov9_annotations"
image_dirs = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets_handsdown/batch"
output_dir = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets_handsdown/annoation_csv_yolov9"

file_counter = 0

# Map class IDs to class names
# Doesnt actually get used. I just like to have it places to read. 

class_map = {
    0: "Head",
    1: "Lower Sensitive",
    2: "Right Arm",
    3: "Left Arm",
    4: "Right Ankle",
    5: "Left Ankle",
    6: "Right Leg",
    7: "Left Leg",
    8: "Torso",
    9: "Threat",
    10: "Upper Sensitive",
    11: 'RIght Wrist', 
    12: 'Right Knee', 
    13: 'Left Shoulder'
}

###########################################################
# Convert each yolo line to a CSV line.
# Inputs - file path, image width and image height
# Outputs - annotation information for the box. 
###########################################################
def yolo_to_csv(file_path: str,  width: int, height: int) -> List[Dict]:
    annotations = []
    attributes = []
    region_count = 0
    
    # open the yolo text file. 
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            
            # convert the yolo box information (center point, height, width)
            # into CSV box information (bottom left point, height, width)
            # class_id, x_center, y_center, w, h = map(float, line.strip().split())
            if len(line.strip().split()) == 5:
                class_id, x_center, y_center, w, h = map(float, line.strip().split())
                x_center *= width
                y_center *= height
                w *= width
                h *= height
                x_min = int(x_center - w / 2)
                y_min = int(y_center - h / 2)

                # CSV annotation information
                annotation = {
                    "name": "rect",
                    "x": int(x_min),
                    "y": int(y_min),
                    "width": int(w),
                    "height": int(h)
                }
            elif len(line.strip().split()) > 5:
                line_split = line.strip().split()
                class_id = line_split.pop(0)

                all_x = []
                all_y = []
                for i in range(0,len(line_split),2):
                    x = line_split.pop(0)
                    y = line_split.pop(0)
                    x = int(float(x) * width)
                    y = int(float(y) * height)
                    all_x.append(x)
                    all_y.append(y)
                
                annotation = {
                    "name": "polygon",
                    "all_points_x": all_x,
                    "all_points_y": all_y
                }

            # label ID
            region_attribute = {"category":{str(int(class_id)):True}}
        
            region_count += 1
            annotations.append(annotation)
            attributes.append(region_attribute)

    if len(annotations) == 0:
        annotations.append({})
        attributes.append({})
    return annotations, attributes, region_count, 

csv_annotations = []
csv_attributes = []
csv_images = []
image_id = 0
row_count = 0

# CSV file fieldnames for the header
field_names = ["filename", 
               "file_size", 
               "file_attributes", 
               "region_count", 
               "region_id", 
               "region_shape_attributes", 
               "region_attributes"]

for folder in os.listdir(annotation_dirs):
    if os.path.isdir(os.path.join(annotation_dirs, folder)) and os.path.isdir(os.path.join(image_dirs, folder)):
        annotation_dir = os.path.join(annotation_dirs, folder, "labels")
        image_dir = os.path.join(image_dirs, folder)
        output_file = os.path.join(output_dir, folder + ".csv")

        # Open a CSV file output file
        with open(output_file, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=field_names)
            writer.writeheader()
            annotation_files = os.listdir(annotation_dir)
            annotation_files.sort()
            # iterate over all files in the YOLO text directory. 
            for filename in annotation_files.copy():
                if filename.endswith('.txt'):
                    
                    annotation_file = os.path.join(annotation_dir, filename)
                    image_file = os.path.join(image_dir, filename.replace('.txt', '.png'))
                    
                    # get image information
                    image = cv2.imread(image_file)
                    height, width = image.shape[:2]
                    size = os.stat(image_file).st_size
                    
                    file_attributes = {}
                    # file_attributes = {"width":width,"height":height}
                    
                    # get the bbox information from the yolo file
                    annotations, attributes, region_count = yolo_to_csv(
                        annotation_file, 
                        width, 
                        height)

                    file_counter += 1
                    region_id = 0
                    # write all the CSV bbox to the CSV
                    for i in range(region_count):
                        row = {"filename":filename.replace('.txt', '.png'), 
                            "file_size":size, 
                            "file_attributes":file_attributes, 
                            "region_count":region_count, 
                            "region_id":i, 
                            "region_shape_attributes":str(annotations[i]).replace("\'","\""), 
                            "region_attributes":str(attributes[i]).replace("\'","\"").replace("True", "true").replace("False","false")}
                        writer.writerow(row)
                    if region_count == 0:
                        row = {"filename":filename.replace('.txt', '.png'), 
                            "file_size":size, 
                            "file_attributes": str({}), 
                            "region_count":region_count, 
                            "region_id":0, 
                            "region_shape_attributes":str({}), 
                            "region_attributes": str({})}
                        writer.writerow(row)
                    image_id += 1

 Batch of CSV to YOLO

In [ ]:
import os
import json
import cv2
import csv
import numpy as np
from typing import List, Dict
from tqdm import tqdm
import shutil

animal = "bear"
annotation_dir = "/media/hdfs1/projects/hdait/fulldata/whole_datasets/14000_and_14TB/labels"
image_dir = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets/batch"
# input_file = "/home/ahufford/Projects/Utils/difficult_contrast.csv"
input_folder = "/media/hdfs1/projects/hdait/fulldata/weapons_training_data_buckets/finished_annotations_qc"

file_counter = 0

isExist = os.path.exists(annotation_dir)
if not isExist:
   os.makedirs(annotation_dir)

# Map class IDs to class names
file_map = {
    "_000.png": "-03.png",
    "_001.png": "-07.png",
    "_002.png": "-11.png",
    "_003.png": "-15.png",
    "_004.png": "-19.png",
    "_005.png": "-23.png",
    "_006.png": "-27.png",
    "_007.png": "-31.png",
    "_008.png": "-35.png",
    "_009.png": "-39.png",
    "_010.png": "-43.png",
    "_011.png": "-47.png",
    "_012.png": "-51.png",
    "_013.png": "-55.png",
    "_014.png": "-59.png",
    "_015.png": "-63.png",
}

class_map = {
    0: "FACE",
    1: "JOINT",
    2: "LIMB",
    3: "EXAMPLE"
}

###########################################################
# Convert each yolo line to a CSV line.
# Inputs -  file path to the CSV file
#           directory to save the yolo files in
#           directory the images are located            
# Outputs - None, the function does the file writing. 
# 
# This function will probably change so I can use the 
# text file write instead of text file append. 
#  
###########################################################
def csv_to_yolo(file_path: str, annotation_dir: str, image_dir: str) -> List[Dict]:
    annotations = []
    attributes = []
    field_names = ["filename", 
               "file_size", 
               "file_attributes", 
               "region_count", 
               "region_id", 
               "region_shape_attributes", 
               "region_attributes"]
    
    region_count = 0
    with open(file_path, newline = '') as f:
        reader = csv.DictReader(f, delimiter=',', quotechar='\"')
        line_count = -1
        xmax = 0
        ymax = 0
        for row in tqdm(reader):
            if line_count == 0:
                header = row
                print(header)
            
            
            else:
                text_filename = row["filename"].replace(".png",".txt").replace(".jpeg",".txt").split('/')[-1]
                with open(os.path.join(annotation_dir, text_filename), "a") as txt_file:
                    if row["region_id"] < row["region_count"]:

                        label_dict = json.loads(row["region_attributes"])
                        shape_dict = json.loads(row["region_shape_attributes"])
                        image_dict = json.loads(row["file_attributes"])
                        
                        try:
                            img_width = image_dict["width"]
                            img_height = image_dict["height"]

                        except KeyError:
                            image_name = os.path.join(image_dir, row["filename"].split('/')[-1])
                            # image_name = image_name.replace(image_name[-8:], file_map[image_name[-8:]])
                            img = cv2.imread(image_name)
                            img_height, img_width, c = img.shape

                        box = None
                        if shape_dict["name"] == "rect":
                            x_min = shape_dict["x"]
                            y_min = shape_dict["y"]
                            
                            
                            w = shape_dict["width"]
                            h = shape_dict["height"]

                            x = int(x_min + w/2.0)
                            y = int(y_min + h/2.0)

                            dw = 1.0 / img_width
                            dh = 1.0 / img_height
                            
                            # Normalization
                            x = x * dw
                            y = y * dh
                            w = w * dw
                            h = h * dh
                                       
                            bbox = [str(x), str(y), str(w), str(h)]

                            label = 0
                            for item in label_dict["category"]:
                                if item.isnumeric():
                                    if label_dict["category"][str(item)]:
                                        label = int(item)
                                        if label == 1:
                                            continue
                                else:
                                    val_list = list(class_map.values())
                                    label = val_list.index(item)
                            box = [str(label)] + bbox
                        
                        # catch the difference in shapes. 
                        if shape_dict["name"] == "polygon":
                            x_points = list(shape_dict["all_points_x"])
                            y_points = list(shape_dict["all_points_y"])

                            label = 0
                            # item = label_dict["category"]
                            
                            # if isinstance(item, dict):
                            #     for something in item:
                            #         if something.isnumeric():
                            #             if label_dict["category"][str(something)]:
                            #                 label = int(something)
                                            
                            #         else:
                            #             val_list = list(class_map.values())
                            #             label = val_list.index(something)
                            
                            # elif item.isnumeric():
                            #     if label_dict["category"][item]:
                            #         label = int(item) 
                            # else:
                            #     val_list = list(class_map.values())
                            #     label = val_list.index(item)
                            bbox = []
                            for i in range(len(x_points)):
                                bbox.append(str(float(x_points[i] / img_width)))
                                bbox.append(str(float(y_points[i] / img_height)))
                            if label == 1:
                                continue
                            label = 0
                            box = [str(label)] + bbox
                        if box:
                            txt_file.write(" ".join(box) + "\n")
                        box = None
            

        
def copy_files_to_folder(list_of_files, destination_folder):
    for f in tqdm(list_of_files):
        try:
            dest_file = os.path.join(destination_folder, f.split('/')[-1])
            if not os.path.exists(dest_file):
                shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

        
            
csv_annotations = []
csv_attributes = []
csv_images = []
image_id = 0
row_count = 0

image_dest_dir = '/media/hdfs1/projects/hdait/fulldata/whole_datasets/14000_and_14TB/images'

for file in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file)
    filename, ext = os.path.splitext(file)
    image_folder = os.path.join(image_dir, filename)
    image_files = [os.path.join(image_folder, image) for image in os.listdir(image_folder) if '.png' in image]
    copy_files_to_folder(image_files, image_dest_dir)
    
    csv_to_yolo(file_path, annotation_dir, image_folder)